<a href="https://colab.research.google.com/github/Swayam17o5/SCT_ML_3/blob/main/dogsCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kaggle


In [2]:
from google.colab import files
uploaded = files.upload()  # Select kaggle.json file

Saving kaggle.json to kaggle.json


In [3]:
import os
import shutil

# Create the .kaggle directory if it doesn't exist
if not os.path.exists('/root/.kaggle'):
    os.makedirs('/root/.kaggle')

# Move the kaggle.json file
shutil.move('/content/kaggle.json', '/root/.kaggle/kaggle.json')

# Set file permissions
os.chmod('/root/.kaggle/kaggle.json', 0o600)

In [4]:
!kaggle datasets download catherinekoo/dogs-vs-cats -d kaggle_data

403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/metadata/catherinekoo/dogs-vs-cats


In [5]:
!kaggle datasets download tongpython/cat-and-dog -d kaggle_data

Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
 61% 132M/218M [00:00<00:00, 1.38GB/s]
100% 218M/218M [00:00<00:00, 679MB/s] 


In [6]:
import shutil
import os

# Create the kaggle_data directory if it doesn't exist
if not os.path.exists('kaggle_data'):
    os.makedirs('kaggle_data')

# Move the downloaded zip file to the correct directory
shutil.move('/content/cat-and-dog.zip', 'kaggle_data/cat-and-dog.zip')

'kaggle_data/cat-and-dog.zip'

In [7]:
import zipfile
import os

# Specify the path to the downloaded zip file
zip_file_path = 'kaggle_data/cat-and-dog.zip'

# Specify the directory where the contents of the zip file should be extracted
extraction_dir = 'kaggle_data/cat-and-dog'

# Create the extraction directory if it doesn't exist
if not os.path.exists(extraction_dir):
    os.makedirs(extraction_dir)

# Open the zip file and extract all contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

print(f"Dataset extracted to: {extraction_dir}")

Dataset extracted to: kaggle_data/cat-and-dog


In [8]:
import os
import shutil
import random

# Define paths
dataset_dir = 'kaggle_data/cat-and-dog'
train_dir = os.path.join(dataset_dir, 'training_set')
test_dir = os.path.join(dataset_dir, 'test_set')

# Create training and testing directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define paths for cat and dog subdirectories
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

# Create cat and dog subdirectories
os.makedirs(train_cats_dir, exist_ok=True)
os.makedirs(train_dogs_dir, exist_ok=True)
os.makedirs(test_cats_dir, exist_ok=True)
os.makedirs(test_dogs_dir, exist_ok=True)

# Define the original image directory within the dataset
original_images_dir = os.path.join(dataset_dir, 'training_set', 'training_set') # The zip file structure is nested

# Get list of image files
image_files = [f for f in os.listdir(original_images_dir) if f.endswith('.jpg')]

# Shuffle the files for random splitting
random.shuffle(image_files)

# Define split ratio
split_ratio = 0.8

# Calculate split index
split_index = int(len(image_files) * split_ratio)

# Split files into training and testing sets
train_files = image_files[:split_index]
test_files = image_files[split_index:]

# Function to move files
def move_files(file_list, destination_dir):
    for filename in file_list:
        src_path = os.path.join(original_images_dir, filename)
        if 'cat' in filename:
            dst_path = os.path.join(destination_dir, 'cats', filename)
        elif 'dog' in filename:
            dst_path = os.path.join(destination_dir, 'dogs', filename)
        else:
            continue # Skip files not named as cat or dog

        # Ensure destination directory exists before moving
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        shutil.move(src_path, dst_path)

# Move training files
move_files(train_files, train_dir)

# Move testing files
move_files(test_files, test_dir)

print("Dataset organized into training and testing directories.")

Dataset organized into training and testing directories.


In [9]:
import cv2
import numpy as np
import os

def preprocess_image(image_path, target_size=(100, 100)):
    """
    Reads, resizes, converts to grayscale, and flattens an image.

    Args:
        image_path (str): The path to the image file.
        target_size (tuple): The desired size of the image (width, height).

    Returns:
        np.ndarray: A flattened numpy array representing the preprocessed image,
                    or None if the image could not be processed.
    """
    try:
        # Read the image in grayscale
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Check if image reading was successful
        if img is None:
            print(f"Warning: Could not read image {image_path}")
            return None

        # Resize the image
        img_resized = cv2.resize(img, target_size)

        # Flatten the image into a 1D array
        img_flattened = img_resized.flatten()

        return img_flattened
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None


In [10]:
# Define paths
dataset_dir = 'kaggle_data/cat-and-dog'
train_dir = os.path.join(dataset_dir, 'training_set')
test_dir = os.path.join(dataset_dir, 'test_set')

# Create lists to store data and labels
train_data = []
train_labels = []
test_data = []
test_labels = []

# Process training data
print("Processing training data...")
for category in ['cats', 'dogs']:
    category_dir = os.path.join(train_dir, category)
    label = 0 if category == 'cats' else 1
    for filename in os.listdir(category_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(category_dir, filename)
            processed_image = preprocess_image(image_path)
            if processed_image is not None:
                train_data.append(processed_image)
                train_labels.append(label)

# Process testing data
print("Processing testing data...")
for category in ['cats', 'dogs']:
    category_dir = os.path.join(test_dir, category)
    label = 0 if category == 'cats' else 1
    for filename in os.listdir(category_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(category_dir, filename)
            processed_image = preprocess_image(image_path)
            if processed_image is not None:
                test_data.append(processed_image)
                test_labels.append(label)

# Convert lists to NumPy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

print("Image preprocessing complete.")
print(f"Training data shape: {train_data.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Testing data shape: {test_data.shape}")
print(f"Testing labels shape: {test_labels.shape}")

Processing training data...
Processing testing data...
Image preprocessing complete.
Training data shape: (0,)
Training labels shape: (0,)
Testing data shape: (0,)
Testing labels shape: (0,)


In [11]:
# Inspect the directory structure to confirm paths
print("Training directory content:")
print(os.listdir(train_dir))
print("\nTesting directory content:")
print(os.listdir(test_dir))

# Also check the content of the cat and dog subdirectories within training
print("\nTraining cats directory content (first 10):")
train_cats_dir_check = os.path.join(train_dir, 'cats')
if os.path.exists(train_cats_dir_check):
    print(os.listdir(train_cats_dir_check)[:10])
else:
    print("Training cats directory not found.")

print("\nTraining dogs directory content (first 10):")
train_dogs_dir_check = os.path.join(train_dir, 'dogs')
if os.path.exists(train_dogs_dir_check):
    print(os.listdir(train_dogs_dir_check)[:10])
else:
    print("Training dogs directory not found.")


Training directory content:
['cats', 'training_set', 'dogs']

Testing directory content:
['cats', 'test_set', 'dogs']

Training cats directory content (first 10):
[]

Training dogs directory content (first 10):
[]


In [12]:
# Define the correct paths to the image directories based on inspection
correct_train_dir = os.path.join(dataset_dir, 'training_set', 'training_set') # Corrected path
correct_test_dir = os.path.join(dataset_dir, 'test_set', 'test_set')       # Corrected path


# Re-create lists to store data and labels
train_data = []
train_labels = []
test_data = []
test_labels = []

# Process training data
print("Processing training data...")
# Assuming the cat images are in a directory named 'cats' and dog images in 'dogs' within correct_train_dir
train_cats_path = os.path.join(correct_train_dir, 'cats')
train_dogs_path = os.path.join(correct_train_dir, 'dogs')

if os.path.exists(train_cats_path):
    for filename in os.listdir(train_cats_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(train_cats_path, filename)
            processed_image = preprocess_image(image_path)
            if processed_image is not None:
                train_data.append(processed_image)
                train_labels.append(0) # 0 for cats
else:
    print(f"Training cats directory not found at {train_cats_path}")

if os.path.exists(train_dogs_path):
    for filename in os.listdir(train_dogs_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(train_dogs_path, filename)
            processed_image = preprocess_image(image_path)
            if processed_image is not None:
                train_data.append(processed_image)
                train_labels.append(1) # 1 for dogs
else:
     print(f"Training dogs directory not found at {train_dogs_path}")


# Process testing data
print("Processing testing data...")
# Assuming the cat images are in a directory named 'cats' and dog images in 'dogs' within correct_test_dir
test_cats_path = os.path.join(correct_test_dir, 'cats')
test_dogs_path = os.path.join(correct_test_dir, 'dogs')

if os.path.exists(test_cats_path):
    for filename in os.listdir(test_cats_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(test_cats_path, filename)
            processed_image = preprocess_image(image_path)
            if processed_image is not None:
                test_data.append(processed_image)
                test_labels.append(0) # 0 for cats
else:
    print(f"Testing cats directory not found at {test_cats_path}")

if os.path.exists(test_dogs_path):
    for filename in os.listdir(test_dogs_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(test_dogs_path, filename)
            processed_image = preprocess_image(image_path)
            if processed_image is not None:
                test_data.append(processed_image)
                test_labels.append(1) # 1 for dogs
else:
    print(f"Testing dogs directory not found at {test_dogs_path}")


# Convert lists to NumPy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

print("Image preprocessing complete.")
print(f"Training data shape: {train_data.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Testing data shape: {test_data.shape}")
print(f"Testing labels shape: {test_labels.shape}")

Processing training data...
Processing testing data...
Image preprocessing complete.
Training data shape: (8005, 10000)
Training labels shape: (8005,)
Testing data shape: (2023, 10000)
Testing labels shape: (2023,)


In [13]:
from sklearn.svm import SVC

# Create an instance of the SVC classifier
# Using default parameters for now, can experiment later
svm_model = SVC()

# Train the SVM model
print("Training SVM model...")
svm_model.fit(train_data, train_labels)
print("SVM model training complete.")

Training SVM model...
SVM model training complete.


In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Make predictions on the test data
predictions = svm_model.predict(test_data)

# Calculate and print accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

# Generate and print classification report
class_report = classification_report(test_labels, predictions)
print("\nClassification Report:")
print(class_report)

# Generate and print confusion matrix
conf_matrix = confusion_matrix(test_labels, predictions)
print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.6500

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.64      0.65      1011
           1       0.65      0.66      0.65      1012

    accuracy                           0.65      2023
   macro avg       0.65      0.65      0.65      2023
weighted avg       0.65      0.65      0.65      2023


Confusion Matrix:
[[644 367]
 [341 671]]
